# Neural Network Chatbot Setup

Notebook ini berisi setup awal untuk membuat chatbot menggunakan neural network dengan NLTK dan TensorFlow.

In [13]:
# Import semua library yang diperlukan
import nltk
from nltk.stem import LancasterStemmer
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import json
import pickle
import random

# Download 'punkt' dari NLTK dengan pengecekan
try:
    nltk.data.find('tokenizers/punkt')
    print("NLTK punkt sudah terinstall")
except LookupError:
    print("Mengunduh NLTK punkt...")
    nltk.download('punkt')
    print("NLTK punkt berhasil diunduh")

# Inisialisasi LancasterStemmer
stemmer = LancasterStemmer()

print("Setup berhasil!")
print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")
print("LancasterStemmer telah diinisialisasi")
print("Semua library siap digunakan")

NLTK punkt sudah terinstall
Setup berhasil!
TensorFlow version: 2.19.0
NumPy version: 2.1.3
LancasterStemmer telah diinisialisasi
Semua library siap digunakan


In [14]:
# Memproses file intents.json
print("Memproses file intents.json...")

# 1. Buka dan muat file 'intents.json'
with open('intents.json') as file:
    data = json.load(file)

# 2. Siapkan list kosong untuk 'words', 'classes', dan 'documents'
words = []
classes = []
documents = []

# Kata-kata yang diabaikan (tanda baca dan karakter khusus)
ignore_words = ['?', '.', ',', '!', "'", '"', ';', ':', '-', '(', ')', '[', ']', '{', '}']

# 3. Lakukan iterasi pada setiap intent di dalam data
for intent in data['intents']:
    # Untuk setiap pattern dalam intent
    for pattern in intent['patterns']:
        # 4. Lakukan tokenisasi kata-kata dari pattern
        tokens = nltk.word_tokenize(pattern)
        
        # Masukkan semua token ke dalam list 'words'
        words.extend(tokens)
        
        # Masukkan pasangan (list token, tag) ke dalam list 'documents'
        documents.append((tokens, intent['tag']))
    
    # 5. Kumpulkan semua 'tag' unik ke dalam list 'classes'
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

# 6. Lakukan stemming pada setiap kata di list 'words'
# Ubah ke huruf kecil, hapus kata-kata yang diabaikan, dan hapus duplikat
words = [stemmer.stem(word.lower()) for word in words if word not in ignore_words]
words = list(set(words))  # Hapus duplikat

# 7. Urutkan list 'words' dan 'classes'
words = sorted(words)
classes = sorted(classes)

# 8. Cetak jumlah dokumen, kelas, dan kata unik untuk verifikasi
print(f"Jumlah dokumen (patterns): {len(documents)}")
print(f"Jumlah kelas (intents): {len(classes)}")
print(f"Jumlah kata unik setelah stemming: {len(words)}")
print(f"Kelas yang ditemukan: {classes}")
print(f"Contoh kata setelah stemming: {words[:10]}")  # Tampilkan 10 kata pertama

Memproses file intents.json...
Jumlah dokumen (patterns): 535
Jumlah kelas (intents): 17
Jumlah kata unik setelah stemming: 462
Kelas yang ditemukan: ['absensi', 'beasiswa', 'cek_nilai', 'goodbye', 'greeting', 'info_dosen', 'jadwal_kuliah', 'jadwal_ujian', 'kontak_admin', 'krs_mata_kuliah', 'pembayaran_spp', 'perpustakaan', 'ruang_kelas', 'skripsi_thesis', 'thanks', 'tugas_pr', 'wifi_internet']
Contoh kata setelah stemming: ['a', 'a101', 'abroad', 'abs', 'absens', 'academ', 'account', 'ad', 'admin', 'adv']


In [15]:
# Mengubah data menjadi data training numerik
print("Mengubah data menjadi format training numerik...")

# 1. Buat list 'training' kosong
training = []

# 2. Buat template output 'output_empty' yang berisi array nol sepanjang jumlah kelas
output_empty = [0] * len(classes)

# 3. Lakukan iterasi pada list 'documents'
for doc in documents:
    # a. Buat 'bag of words' (list nol sepanjang jumlah kata unik)
    bag = [0] * len(words)
    
    # Ambil pattern (token) dan tag dari dokumen
    pattern_words = doc[0]
    tag = doc[1]
    
    # Stem setiap kata dalam pattern
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    # b. Untuk setiap kata dalam pattern dokumen, tandai '1' pada posisi yang sesuai di 'bag'
    for word in pattern_words:
        if word in words:
            bag[words.index(word)] = 1
    
    # c. Buat 'output_row' dengan menyalin 'output_empty' lalu menandai '1' pada posisi tag
    output_row = output_empty[:]  # Salin template
    output_row[classes.index(tag)] = 1
    
    # 4. Masukkan pasangan [bag, output_row] ke dalam list 'training'
    training.append([bag, output_row])

# 5. Acak (shuffle) list 'training' dan ubah menjadi NumPy array
random.shuffle(training)
training = np.array(training, dtype=object)

# 6. Pisahkan array tersebut menjadi 'train_x' (fitur) dan 'train_y' (label)
train_x = list(training[:, 0])  # Kolom pertama: bag of words
train_y = list(training[:, 1])  # Kolom kedua: output labels

# Konversi ke NumPy array dengan tipe float
train_x = np.array(train_x, dtype=np.float32)
train_y = np.array(train_y, dtype=np.float32)

print("Data training berhasil dibuat!")
print(f"Ukuran train_x: {train_x.shape}")
print(f"Ukuran train_y: {train_y.shape}")
print(f"Contoh bag of words pertama: {train_x[0]}")
print(f"Contoh output label pertama: {train_y[0]}")
print(f"Total training samples: {len(train_x)}")

Mengubah data menjadi format training numerik...
Data training berhasil dibuat!
Ukuran train_x: (535, 462)
Ukuran train_y: (535, 17)
Contoh bag of words pertama: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [16]:
# Membuat dan melatih model Neural Network
print("Membangun model Neural Network...")

# 1. Buat model Sequential dari Keras
model = Sequential()

# 2. Tambahkan layer secara berurutan
# Dense layer dengan 128 neuron, input_shape sesuai panjang data training, dan aktivasi 'relu'
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))

# Dropout layer dengan rate 0.5
model.add(Dropout(0.5))

# Dense layer dengan 64 neuron dan aktivasi 'relu'
model.add(Dense(64, activation='relu'))

# Dropout layer dengan rate 0.5
model.add(Dropout(0.5))

# Output Dense layer dengan jumlah neuron sesuai jumlah kelas dan aktivasi 'softmax'
model.add(Dense(len(train_y[0]), activation='softmax'))

# 3. Compile model
model.compile(loss='categorical_crossentropy', 
             optimizer='sgd', 
             metrics=['accuracy'])

# Tampilkan arsitektur model
print("Arsitektur model:")
model.summary()

print("\nMemulai training model...")

# 4. Latih model (model.fit) selama 200 epoch dengan batch_size 5
history = model.fit(train_x, train_y, 
                   epochs=200, 
                   batch_size=5, 
                   verbose=1)

print("\nTraining selesai!")
print(f"Akurasi akhir: {history.history['accuracy'][-1]:.4f}")
print(f"Loss akhir: {history.history['loss'][-1]:.4f}")

Membangun model Neural Network...
Arsitektur model:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │        59,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 17)             │         1,105 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 68,625 (268.07 KB)

 Trainable params: 68,625 (268.07 KB)

 Non-trainable params: 0 (0.00 B)


Memulai training model...
Epoch 1/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0503 - loss: 2.8400
Epoch 2/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1041 - loss: 2.8067
Epoch 3/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1420 - loss: 2.7754
Epoch 4/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1754 - loss: 2.7350
Epoch 5/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1546 - loss: 2.7447
Epoch 6/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1384 - loss: 2.7450
Epoch 7/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1462 - loss: 2.7370
Epoch 8/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1570 - loss: 2.6945
Epoch 9/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2221 - loss: 2.6326
Epoch 10/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2117 - loss: 2.6274
Epoch 11/200
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2134 - loss: 2.6358
Epoch 12/200
1

In [17]:
# Menyimpan model dan data yang sudah dilatih
print("Menyimpan model dan data...")

# Simpan model yang sudah dilatih ke file 'chatbot_model.h5'
model.save('chatbot_model.h5')
print("✓ Model berhasil disimpan ke 'chatbot_model.h5'")

# Simpan list 'words' dan 'classes' ke dalam file 'data.pickle' menggunakan pickle
with open('data.pickle', 'wb') as file:
    pickle.dump({'words': words, 'classes': classes}, file)
print("✓ Data words dan classes berhasil disimpan ke 'data.pickle'")

print("\n🎉 Proses training dan penyimpanan selesai!")
print("File yang telah dibuat:")
print("- chatbot_model.h5 (Model neural network)")
print("- data.pickle (Vocabulary dan classes)")
print("\nModel chatbot siap digunakan!")

Menyimpan model dan data...
✓ Model berhasil disimpan ke 'chatbot_model.h5'
✓ Data words dan classes berhasil disimpan ke 'data.pickle'

🎉 Proses training dan penyimpanan selesai!
File yang telah dibuat:
- chatbot_model.h5 (Model neural network)
- data.pickle (Vocabulary dan classes)

Model chatbot siap digunakan!


In [19]:
# Fungsi-fungsi untuk prediksi dan respons chatbot
print("Membuat fungsi-fungsi untuk prediksi...")

def clean_up_sentence(sentence):
    """
    Fungsi untuk melakukan tokenisasi dan stemming pada kalimat input
    """
    # Tokenisasi kalimat
    sentence_words = nltk.word_tokenize(sentence)
    
    # Stemming setiap kata dan ubah ke huruf kecil
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    
    return sentence_words

def bow(sentence, words):
    """
    Fungsi untuk mengubah kalimat menjadi bag-of-words berdasarkan vocabulary
    """
    # Bersihkan kalimat input
    sentence_words = clean_up_sentence(sentence)
    
    # Buat bag dengan panjang sesuai vocabulary
    bag = [0] * len(words)
    
    # Tandai 1 untuk kata yang ada dalam vocabulary
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                bag[i] = 1
    
    # Konversi ke numpy array
    return np.array(bag, dtype=np.float32)

def predict_class(sentence, model):
    """
    Fungsi untuk memprediksi intent dari kalimat menggunakan model
    """
    # Buat bag of words dari kalimat
    p = bow(sentence, words)
    
    # Reshape untuk input model (batch dimension)
    p = p.reshape(1, -1)
    
    # Prediksi menggunakan model
    res = model.predict(p)[0]
    
    # Threshold probabilitas minimum
    ERROR_THRESHOLD = 0.25
    
    # Filter hasil prediksi di atas threshold
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    
    # Urutkan berdasarkan probabilitas (tertinggi dulu)
    results.sort(key=lambda x: x[1], reverse=True)
    
    # Buat list intent dengan probabilitas
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    
    return return_list

def getResponse(ints, intents_json):
    """
    Fungsi untuk memilih respons acak dari intent dengan probabilitas tertinggi
    """
    if len(ints) == 0:
        return "Maaf, saya tidak mengerti. Bisakah Anda mengulanginya?"
    
    # Ambil tag intent dengan probabilitas tertinggi
    tag = ints[0]['intent']
    
    # Cari intent dalam data JSON
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            # Pilih respons secara acak dari list responses
            result = random.choice(i['responses'])
            break
    else:
        result = "Maaf, saya tidak dapat memberikan respons untuk pertanyaan tersebut."
    
    return result

print("✓ Fungsi prediksi berhasil dibuat:")
print("- clean_up_sentence(): tokenisasi dan stemming")
print("- bow(): konversi ke bag-of-words")
print("- predict_class(): prediksi intent dengan model")
print("- getResponse(): pemilihan respons acak")
print("\nFungsi-fungsi siap digunakan untuk chatbot!")

Membuat fungsi-fungsi untuk prediksi...
✓ Fungsi prediksi berhasil dibuat:
- clean_up_sentence(): tokenisasi dan stemming
- bow(): konversi ke bag-of-words
- predict_class(): prediksi intent dengan model
- getResponse(): pemilihan respons acak

Fungsi-fungsi siap digunakan untuk chatbot!


In [21]:
# Menjalankan chatbot secara interaktif
print("Memuat model dan data untuk chatbot...")

# 1. Muat kembali model dari file 'chatbot_model.h5'
from tensorflow.keras.models import load_model
chatbot_model = load_model('chatbot_model.h5')
print("✓ Model berhasil dimuat dari 'chatbot_model.h5'")

# 2. Muat kembali 'words' dan 'classes' dari file 'data.pickle'
with open('data.pickle', 'rb') as file:
    data_loaded = pickle.load(file)
    words_loaded = data_loaded['words']
    classes_loaded = data_loaded['classes']
print("✓ Data words dan classes berhasil dimuat dari 'data.pickle'")

# 3. Muat kembali data dari 'intents.json'
with open('intents.json') as file:
    intents_data = json.load(file)
print("✓ Data intents berhasil dimuat dari 'intents.json'")

# 4. Buat fungsi 'chatbot_response(text)' yang mengoordinasikan prediksi dan respons
def chatbot_response(text):
    """
    Fungsi untuk mendapatkan respons chatbot dari input text
    """
    # Prediksi intent dari input text
    ints = predict_class(text, chatbot_model)
    
    # Dapatkan respons berdasarkan intent yang diprediksi
    response = getResponse(ints, intents_data)
    
    return response

print("\n🤖 Chatbot Neural Network siap digunakan!")
print("PENTING: Untuk menghentikan chatbot, ketik 'quit' atau 'exit'")
print("Atau gunakan Kernel > Interrupt untuk menghentikan paksa")
print("=" * 50)

# 5. Fungsi untuk menjalankan chatbot dengan handling error yang lebih baik
def run_chatbot():
    try:
        while True:
            # Meminta input dari pengguna
            user_input = input("Anda: ")
            
            # 6. Jika pengguna mengetik 'quit' atau 'exit', hentikan loop
            if user_input.lower() in ['quit', 'exit', 'stop', 'keluar']:
                print("Bot: Terima kasih telah menggunakan chatbot! Sampai jumpa! 👋")
                break
            
            # Skip jika input kosong
            if not user_input.strip():
                continue
            
            # 7. Untuk setiap input, panggil fungsi 'chatbot_response' dan cetak balasan
            bot_response = chatbot_response(user_input)
            print(f"Bot: {bot_response}")
            print("-" * 30)
            
    except KeyboardInterrupt:
        print("\n\nBot: Chatbot dihentikan oleh pengguna. Sampai jumpa! 👋")
    except Exception as e:
        print(f"\nTerjadi error: {e}")
        print("Chatbot dihentikan.")

# Jalankan chatbot
print("Memulai chatbot... (Tekan Ctrl+C atau ketik 'quit' untuk berhenti)")
run_chatbot()

Memuat model dan data untuk chatbot...
✓ Model berhasil dimuat dari 'chatbot_model.h5'
✓ Data words dan classes berhasil dimuat dari 'data.pickle'
✓ Data intents berhasil dimuat dari 'intents.json'

🤖 Chatbot Neural Network siap digunakan!
PENTING: Untuk menghentikan chatbot, ketik 'quit' atau 'exit'
Atau gunakan Kernel > Interrupt untuk menghentikan paksa
Memulai chatbot... (Tekan Ctrl+C atau ketik 'quit' untuk berhenti)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Bot: Yooo! Gimana kuliahnya? Ada yang perlu ditanyain?
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Bot: Yooo! Gimana kuliahnya? Ada yang perlu ditanyain?
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Bot: Halo kak! Ada pertanyaan tentang jadwal, nilai, atau yang lain?
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Bot: Halo kak! Ada pertanyaan tentang jadwal, nilai, atau yang lain?
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Bot: Nilai